In [14]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

In [15]:
# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

In [16]:
# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]

In [17]:
# Given order
order = [
    "Актуализация", "Внебаланс (new)", "ДИЗ", "ДИЗ (возражения)", "ДИЗ (чередование)",
    "Коллекшн ранний сбор", "Матрица мотиваторов", "Матрица мотиваторов (распознавание эмоций)",
    "Матрица мотиваторов Хард. Ранний сбор", "Матрица мотиваторов Хард. Ранний сбор (кредитные карты)",
    "Матрица мотивации. Выезд", "Мультидоговоры. Универсальный", "Неконтактные", "Неконтактные (внебаланс)",
    "Неконтактные (поздний сбор)", "Неконтактные Суд", "Ожидание выезда", "Ожидание суда",
    "Подготовка к передаче в КА", "Преколлекшн (rename)", "Суд", "Усиление ПС Предцессия", 
    "Усиление позднего сбора", "ФЗ-230"
]
# Given column order
column_order = [
    "АО: Абонент не отвечает", "АО: Абонент недоступен", "АО: Номер не существует", 
    "АО: Нужен внутренний номер", "АО: Соединение установлено", "АО: Умные голосовые помощники", 
    "АО: Факс", "Должник молчит", "Должник не уверен", "Должник неизвестен", "Должник умер", 
    "Запрос реструктуризации", "Заявление о факте платежа", "Клиент болен", "Не передадут информацию", 
    "Не пройдена верификация", "Неконструктивный диалог", "Нужна помощь оператора", "Нужна помощь оператора *", 
    "Обещание оплатить", "Обещание частичной оплаты", "Оплата по испол.листу", "Отказ от верификации", 
    "Отказ от оплаты", "Отрицает долг", "Перезвонить", "Просьба передать информацию", "Сброс звонка роботом", 
    "Связь прервалась"
]

In [18]:

# 1. Filter the dataframe
filtered_df = df[df['Результат автооценки'] != 100] # !!!
#filtered_df = df # !!!
# 2. Create the pivot table
pivot_table = filtered_df.pivot_table(index='Имя колл-листа', 
                                      columns='Результат робота', 
                                      values='Результат автооценки', 
                                      aggfunc='size', 
                                      fill_value=0)

'''pivot_table = filtered_df.pivot_table(index='Имя колл-листа', 
                                      columns='Результат робота', 
                                      values='Результат автооценки', 
                                      aggfunc='count', 
                                      fill_value=0)'''

pivot_table_mean = df.pivot_table(index='Имя колл-листа', columns='Результат робота', values='Результат автооценки', aggfunc='mean')


'''# 3. Ensure all desired columns are present
for col in column_order:
    if col not in pivot_table.columns:
        pivot_table[col] = 0

# Ensure all desired columns are present in the mean pivot table
for col in column_order:
    if col not in pivot_table_mean.columns:
        pivot_table_mean[col] = 0'''


'''# Reorder columns based on the given order
pivot_table = pivot_table[column_order]
pivot_table_mean = pivot_table_mean[column_order]
# Assuming pivot_table is the pivot table you previously created
pivot_table = pivot_table[column_order]
pivot_table = pivot_table.reindex(order)
pivot_table_mean = pivot_table_mean.reindex(order)'''

'# Reorder columns based on the given order\npivot_table = pivot_table[column_order]\npivot_table_mean = pivot_table_mean[column_order]\n# Assuming pivot_table is the pivot table you previously created\npivot_table = pivot_table[column_order]\npivot_table = pivot_table.reindex(order)\npivot_table_mean = pivot_table_mean.reindex(order)'

In [19]:
pivot_table

Результат робота,АО: Умные голосовые помощники,Должник не уверен,Должник неизвестен,Запрос реструктуризации,Заявление о факте платежа,Клиент болен,Не передадут информацию,Не пройдена верификация,Неконструктивный диалог,Нужна помощь оператора,Нужна помощь оператора *,Обещание оплатить,Обещание частичной оплаты,Оплата по испол.листу,Отказ от верификации,Отказ от оплаты,Отрицает долг,Перезвонить,Просьба передать информацию,Связь прервалась
Имя колл-листа,,,,,,,,,,,,,,,,,,,,
Актуализация,1,0,1,0,0,1,4,1,0,1,0,0,0,0,1,0,0,0,4,0
Внебаланс (new),20,0,56,0,9,6,47,1,92,86,0,80,29,6,9,144,5,14,90,0
ДИЗ,9,0,13,0,1,1,7,0,1,22,0,9,0,4,8,28,2,5,12,1
ДИЗ (возражения),1,0,2,0,0,0,1,0,0,4,0,1,0,0,0,4,0,0,2,0
ДИЗ (чередование),0,0,2,0,0,1,0,0,0,0,0,1,0,0,0,4,0,0,0,0
Коллекшн ранний сбор,1,0,14,0,23,1,11,10,0,11,0,52,0,0,19,16,2,4,30,4
Матрица мотиваторов,6,0,27,5,88,6,48,0,57,127,5,1665,0,0,29,460,50,19,110,3
Матрица мотиваторов (распознавание эмоций),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
Матрица мотиваторов Хард. Ранний сбор,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [20]:
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill
import xlsxwriter


def format_xlsx(pivot_table:pd.core.frame.DataFrame, sheet:str = 'Отчет 1', name:str = "pivot_table_gradient_colorscale.xlsx"):
    # Create an Excel writer and export the pivot table to an Excel file
    excel_file_path = name
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        pivot_table.to_excel(writer, sheet_name=sheet, index=True)

        # Get the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets[sheet]

        # Get the dimensions of the pivot table
        max_row = len(pivot_table)
        max_col = len(pivot_table.columns)

        # Add a format for the header cells
        header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'border': 1, 'bg_color': '#EFEFEF', 'align': 'center'})

        # Set the column width and format for the header
        for col_num, value in enumerate(pivot_table.columns.values):
            worksheet.write(0, col_num + 1, value, header_format)
            column_len = max(pivot_table[value].astype(str).str.len().max(), len(value)) + 2
            worksheet.set_column(col_num + 1, col_num + 1, column_len)
        
        # Add a format for the value cells
        value_format = workbook.add_format({'border': None, 'valign': 'vcenter', 'align': 'center',})

        # Get Max and Min of the value FFS
        max_value = pivot_table.max().max()
        min_value = pivot_table.min().min()


        # Apply gradient color scale to value cells
        worksheet.conditional_format(1, 1, max_row, max_col, {
            'type': '3_color_scale',
            'min_color': '#A6D86E',  # Green
            'mid_color': '#FFFFFE',  # White (for NaN)
            'max_color': '#e85f5f',  # Red
            'min_type': 'num',
            'mid_type': 'num',
            'max_type': 'num'
        })
        print(f'file: {name} -- Transformed 0')

In [21]:
format_xlsx(pivot_table)

PermissionError: [Errno 13] Permission denied: 'pivot_table_gradient_colorscale.xlsx'

# Динамика по кол-листам + статусам + 3 в 1

In [141]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv','./Reports/04-08.csv','./Reports/05-08.csv','./Reports/06-08.csv','./Reports/07-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]


In [142]:
df['Ошибки'] = df['Результат автооценки'] != 100
df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
df = df.reset_index(drop=True)

In [143]:
# Mask
df_mistakes = df[df['Результат автооценки'] != 100] # !!!

In [144]:
# Add numbers to index
# Add numbers to index
def update_index(dataframe):
    new_index = [f'{i if len(str(i)) > 1 else f"0{i}"} {row}' for i, row in enumerate(dataframe.index)]
    dataframe.index = new_index
    return dataframe


# Create dynamic Mistakes count (1)
pivot_df_mistakes = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Ошибки', aggfunc='sum')
pivot_df_mistakes = pivot_df_mistakes.fillna(0)
pivot_df_mistakes = pivot_df_mistakes.replace(0.00, '')
pivot_df_mistakes.columns = pd.to_datetime(pivot_df_mistakes.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_mistakes = pivot_df_mistakes.sort_index(axis=1)  # Fix date Time
tmp_pivot_df_mistakes = pivot_df_mistakes.copy()  # Fix %%
pivot_df_mistakes.index = pivot_df_mistakes.index + ' (ошибки шт.)'
pivot_df_mistakes = update_index(pivot_df_mistakes)
# Create dynamic Calls count (2)
pivot_df_calls = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='count', fill_value=0)
pivot_df_calls.columns = pd.to_datetime(pivot_df_calls.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_calls = pivot_df_calls.sort_index(axis=1)  # Fix date Time
tmp_pivot_df_calls = pivot_df_calls.copy()  # Fix %%
pivot_df_calls.index = pivot_df_calls.index + ' (всего шт.)'
pivot_df_calls = update_index(pivot_df_calls)
# Create dynamic Mean Autoscore (3)
pivot_df_mean = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='mean', fill_value='')
pivot_df_mean.columns = pd.to_datetime(pivot_df_mean.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_mean = pivot_df_mean.sort_index(axis=1)  # Fix date Time
pivot_df_mean.index = pivot_df_mean.index + ' (средняя АО)'
pivot_df_mean = update_index(pivot_df_mean)
# Create dynamic Error Percentage (4)
pivot_df_mistakes_filled = tmp_pivot_df_mistakes.replace('', 0)
pivot_df_error_rate = (pivot_df_mistakes_filled / tmp_pivot_df_calls).applymap(lambda x: x if not pd.isna(x) else '')
pivot_df_error_rate.index = pivot_df_error_rate.index + ' (доля ошибок %)'
pivot_df_error_rate = update_index(pivot_df_error_rate)
# Create Mega-Pivot
# Concatenate the pivot tables vertically along rows (axis=0)
mega_pivot = pd.concat([pivot_df_mistakes, pivot_df_calls, pivot_df_mean, pivot_df_error_rate], axis=0)
mega_pivot = mega_pivot.sort_index()


In [145]:
mega_pivot

Дата,2023-07-29,2023-07-30,2023-08-01,2023-08-04,2023-08-05,2023-08-06,2023-08-07
00 Актуализация (всего шт.),3760,4025,3744,1404,4422,3305,3097
00 Актуализация (доля ошибок %),0.00212766,0.00149068,0.00133547,0.00356125,0.00135685,0.00272315,0.00161447
00 Актуализация (ошибки шт.),8,6,5,5,6,9,5
00 Актуализация (средняя АО),99.9548,99.9652,99.9746,99.9217,99.9683,99.9334,99.9629
01 Внебаланс (new) (всего шт.),78888,78295,62343,65957,74438,75833,64264
...,...,...,...,...,...,...,...
23 Усиление ПС Предцессия (средняя АО),,100,,,,,
24 ФЗ-230 (всего шт.),96820,101159,3,0,0,0,0
24 ФЗ-230 (доля ошибок %),0.00582524,0.00671221,0,,,,
24 ФЗ-230 (ошибки шт.),564,679,,,,,


In [146]:
error_rate_rows = mega_pivot[mega_pivot.index.str.contains("(доля ошибок %)")]
errors_rows = mega_pivot[mega_pivot.index.str.contains("(ошибки шт.)")]

c:\Users\Alex\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [147]:
error_rate_rows.apply(pd.to_numeric, errors='coerce').max().max()
error_rate_rows.apply(pd.to_numeric, errors='coerce').min().min()

errors_rows.apply(pd.to_numeric, errors='coerce').max().max()
errors_rows.apply(pd.to_numeric, errors='coerce').min().min()

1.0

Excel Shit

In [148]:
import pandas as pd

# Find min/max percent
max_percent = error_rate_rows.apply(pd.to_numeric, errors='coerce').max().max()
min_percent = error_rate_rows.apply(pd.to_numeric, errors='coerce').min().min()
# Find min/max mistakes
max_errors = errors_rows.apply(pd.to_numeric, errors='coerce').max().max()
min_errors = errors_rows.apply(pd.to_numeric, errors='coerce').min().min()
# Specify the Excel file path
excel_file_path = 'output.xlsx'

# Create a color scale conditional formatting rule
color_scale_rule_percent = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_percent,
                'mid_type': 'percentile',
                'mid_value': 75,
                'max_type': 'num',
                'max_value': max_percent
                }

# Create a color scale conditional formatting rule
color_scale_rule_errors = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_errors,
                'mid_type': 'percentile',
                'mid_value': 75,
                'max_type': 'num',
                'max_value': max_errors
                }

# Create a Pandas Excel writer using xlsxwriter as the engine
writer = pd.ExcelWriter(excel_file_path, engine='xlsxwriter')

# Write the DataFrame to the Excel file
mega_pivot.to_excel(writer, sheet_name='Все звонки')

# Access the xlsxwriter workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Все звонки']

for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_percent)
# Define a percentage format
percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF'})

for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        worksheet.set_row(row_num+1, None, percentage_format)


for row_num, row in enumerate(mega_pivot.index):
    if "(ошибки шт.)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_errors)

# Custom index format
index_format = workbook.add_format(
    {'bold': True, 'border': 1, 'bg_color': '#FFFFFF', 'align': 'left'})
for row_num, row in enumerate(mega_pivot.index):
    worksheet.write(f'A{row_num+2}', row, index_format)

# Define a white fill format
white_fill_format = workbook.add_format({'bg_color': '#FFFFFF', 'border': 0})

# Apply the white background to the entire worksheet
worksheet.set_column(0, 0, 60, white_fill_format)
worksheet.set_column(1, 100, 18, white_fill_format)
'''# Custom header format
header_format = workbook.add_format(
                {'bold': True, 'border': 1, 'bg_color': '#AFEFEF','align': 'center'})
for col_num, value in enumerate(mega_pivot.columns.values):
                worksheet.write(0, col_num + 1, str(value)[:-19], header_format)'''
# Save the Excel file
writer.save()

In [134]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.styles import PatternFill

df = mega_pivot

# Specify the Excel file path
excel_file_path = 'output.xlsx'

# Create a Pandas Excel writer
writer = pd.ExcelWriter(excel_file_path, engine='openpyxl')

# Write the DataFrame to the Excel file
df.to_excel(writer, sheet_name='Sheet1')

# Access the openpyxl workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Create a color scale conditional formatting rule
color_scale_rule = ColorScaleRule(
    start_type="min",
    start_color="A6D86E",  # Green for minimum value (0%)
    mid_type="percentile",
    mid_value=70,
    mid_color="FFFF00",    # Yellow for mid-value (50%)
    end_type="max",
    end_color="e85f5f"     # Red for maximum value (100%)
)

'''# Apply the color scale conditional formatting to the relevant range of cells
for row_num, row in enumerate(df.index):
    if "(доля ошибок %)" in row:
        format_range = 'B' + str(row_num + 2) + ':AR' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_formatting.add(format_range, color_scale_rule)'''

# Save the Excel file
writer.save()


In [104]:
'''
pivot_df_mistakes
pivot_df_calls
pivot_df_mean
pivot_df_error_rate
'''
def update_index(dataframe):
    new_index = [f'{i} {row}' for i, row in enumerate(dataframe.index)]
    dataframe.index = new_index
    return dataframe

# Create a new index with numbers added
new_index = [f'{i} {row}' for i, row in enumerate(pivot_df_error_rate.index)]

# Assign the new index to pivot_df_error_rate
pivot_df_error_rate.index = new_index

# Print the modified DataFrame
pivot_df_error_rate

Дата,2023-07-29,2023-07-30,2023-08-01,2023-08-04,2023-08-05,2023-08-06,2023-08-07
0 00 Актуализация (доля ошибок %),0.00212766,0.00149068,0.00133547,0.00356125,0.00135685,0.00272315,0.00161447
1 01 Внебаланс (new) (доля ошибок %),0.00158452,0.00203078,0.00295141,0.00447261,0.0033988,0.00258463,0.00287875
2 02 ДИЗ (доля ошибок %),0.00144033,0.0014802,0.00270333,0.00288443,0.00258095,0.00157293,0.00235113
3 03 ДИЗ (возражения) (доля ошибок %),0.000848536,0.00129842,0,0.00455298,0.00171282,0.000506586,0.00140885
4 04 ДИЗ (чередование) (доля ошибок %),0.000749344,0.000787712,,0.00226244,0.00159151,0.000972763,0.00110072
5 05 Исполнительное производство (доля ошибок %),,,,,,,0.00357472
6 06 Исполнительное производство (тест контур API) (доля ошибок %),,,,,,,0
7 07 Коллекшн ранний сбор (доля ошибок %),0.00303375,0.0034904,0.0061423,0.00573428,0.0050576,0.00256944,0.0039343
8 08 Матрица мотиваторов (доля ошибок %),0.00791906,0.00487552,0.00966084,0.0381433,0.0450178,0.0138448,0.0242188
9 09 Матрица мотиваторов (распознавание эмоций) (доля ошибок %),0,0,0.0107527,0.0113636,0.00571429,0.00325203,0.00507614


In [48]:
# List of the pivot tables
pivot_tables = [pivot_df_mistakes, pivot_df_calls, pivot_df_mean, pivot_df_error_rate]

# Concatenate the pivot tables vertically along rows (axis=0)
merged_pivot = pd.concat(pivot_tables, axis=0)
merged_pivot

Дата,2023-07-29,2023-07-30,2023-08-01,2023-08-04,2023-08-05,2023-08-06,2023-08-07
Имя колл-листа,,,,,,,
Актуализация,8,6,5,5,6,9,5
Актуализация,99.9548,99.9652,99.9746,99.9217,99.9683,99.9334,99.9629
Актуализация,0.00212766,0.00149068,0.00133547,0.00356125,0.00135685,0.00272315,0.00161447
Актуализация,3760,4025,3744,1404,4422,3305,3097
Внебаланс (new),0.00158452,0.00203078,0.00295141,0.00447261,0.0033988,0.00258463,0.00287875
...,...,...,...,...,...,...,...
Усиление ПС Предцессия,,,,,,,
ФЗ-230,96820,101159,3,0,0,0,0
ФЗ-230,564,679,,,,,


In [34]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]
df['Ошибки'] = df['Результат автооценки'] != 100
df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
df = df.reset_index(drop=True)
df_mistakes = df[df['Результат автооценки'] != 100] # !!!

In [35]:
pivot_df = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Ошибки', aggfunc='sum')
# If you want to handle missing values (NaN) gracefully, you can replace them with 0
pivot_df = pivot_df.fillna(0)
pivot_df = pivot_df.replace(0.00, '')
pivot_df

Дата                                         02.08.2023 03.08.2023 31.07.2023
Имя колл-листа Результат робота                                              
Актуализация   АО: Абонент недоступен                                        
               АО: Номер не существует                                       
               АО: Нужен внутренний номер                                    
               АО: Соединение установлено                                    
               АО: Умные голосовые помощники                              1.0
...                                                 ...        ...        ...
ФЗ-230         Отрицает долг                                             18.0
               Перезвонить                                               13.0
               Просьба передать информацию                               50.0
               Сброс звонка роботом                                          
               Связь прервалась                                           1.0

[439 rows x 3 columns]

In [36]:
# Assuming you have a dataframe named 'df'
# Creating a pivot table to calculate the count of Ошибки (True values) and the count of Результат робота
pivot_df = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Ошибки', aggfunc='sum')

# Calculate the count of Результат робота for each combination
count_df = df.groupby(['Имя колл-листа', 'Результат робота', 'Дата'])['Результат робота'].count().unstack()

# Calculating the fraction as a string
pivot_df = (pivot_df / count_df).applymap(lambda x: x if not pd.isna(x) else '')

pivot_df

Дата                                         02.08.2023 03.08.2023 31.07.2023
Имя колл-листа Результат робота                                              
Актуализация   АО: Абонент недоступен               0.0        0.0        0.0
               АО: Номер не существует                         0.0        0.0
               АО: Нужен внутренний номер           0.0        0.0        0.0
               АО: Соединение установлено           0.0        0.0        0.0
               АО: Умные голосовые помощники        0.0        0.0   0.002703
...                                                 ...        ...        ...
ФЗ-230         Отрицает долг                                         0.147541
               Перезвонить                                           0.036827
               Просьба передать информацию          0.0        0.0    0.02004
               Сброс звонка роботом                                       0.0
               Связь прервалась                     0.0        0.0   0.000108

[439 rows x 3 columns]